In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [7]:
#x,y,c,
Coords = np.load("RootCoordinates/Demi-PlieX.npy")
print(Coords)
print(len(Coords))

[[0.502016 0.271066 0.896372 ... 0.497905 0.709778 0.839966]
 [0.502061 0.268482 0.890618 ... 0.497959 0.709911 0.81064 ]
 [0.500166 0.268516 0.903899 ... 0.498042 0.70719  0.806747]
 ...
 [0.499997 0.311957 0.857867 ... 0.495855 0.70155  0.844654]
 [0.499946 0.314676 0.875749 ... 0.495815 0.696237 0.858081]
 [0.499965 0.314577 0.876629 ... 0.495873 0.696059 0.871991]]
869


In [8]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [9]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/Demi-PlieMeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/Demi-PlieMeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.49807198 0.49855715 0.46426527 0.44007892 0.44986212 0.53243978
 0.55690166 0.55183423 0.5        0.47689986 0.44626658 0.48549923
 0.52318372 0.55629868 0.5198698  0.49047708 0.50539987 0.48057693
 0.51566911 0.54819716 0.54532098 0.51166811 0.45877543 0.45907851
 0.49317074]
25
[-0.28829483 -0.3583691  -0.35860922 -0.43339914 -0.49472028 -0.35752859
 -0.43044281 -0.49308661 -0.5        -0.50048314 -0.60960887 -0.70430716
 -0.49962094 -0.60744698 -0.7041428  -0.28080148 -0.28022812 -0.29188414
 -0.29093084 -0.72869719 -0.71935891 -0.71175603 -0.72912958 -0.72220953
 -0.71181121]
25
[(0.49807198273877945, -0.28829482968929815), (0.49855715420023006, -0.3583691047180671), (0.4642652692750292, -0.3586092186421173), (0.4400789159953972, -0.43339914154200215), (0.4498621150747981, -0.4947202784810131), (0.5324397779056391, -0.35752859378596086), (0.5569016570770996, -0.4304428089758345), (0.5518342278481011, -0.49308660989643244), (0.5, -0.5), (0.4768998607594935, -0.5004831426927498), 

In [10]:
POSE = "Demi-Plie"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.01767449701178634, 0.0058788860173163105, 0.005989079250787183, 0.01101388390786606, 0.003104328047403033, 0.005337869591611265, 0.012623886147763785, 0.005480277125064207, 0.0, 0.0022348991956899437, 0.010668025334259642, 0.003439640845995146, 0.0053259926814719204, 0.011347729532060752, 0.0005880319757295073, 0.016880495418867953, 0.016110466409581768, 0.01564205226199344, 0.01198043482079273, 0.00567307197094395, 0.007681443543429, 0.006644195279420146, 0.005225133399081295, 0.006195304957286167, 0.005152398206223791], [0.020210405216309922, 0.008445099350472572, 0.011123611955122103, 0.012830556869510719, 0.00832663328645602, 0.00778289080520077, 0.012661747437649038, 0.005095176393101771, 0.0, 0.0007792955646917263, 0.009748418315756966, 0.003459189708671254, 0.0040542229326218015, 0.00919740107560045, 0.0025604169958256397, 0.019257436443540152, 0.019279571899176326, 0.018202119361207903, 0.014524115133661739, 0.002952200079047388, 0.0055752721885554465, 0.0032301228456602478

In [11]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.502016, 0.2710660000000001, 0.896372, 0.4999800000000001, 0.3526650000000001, 0.9428300000000001, 0.46526399999999996, 0.3527040000000001, 0.9068849999999999, 0.43473599999999996, 0.42376800000000003, 0.882987, 0.448843, 0.491788, 0.8760089999999999, 0.534696, 0.3526910000000001, 0.8889600000000001, 0.565317, 0.4210330000000001, 0.9382889999999999, 0.5571539999999999, 0.49177000000000004, 0.83789, 0.5, 0.5, 0.812383, 0.477538, 0.5026250000000001, 0.762115, 0.436708, 0.6143460000000001, 0.9121309999999999, 0.48773, 0.701689, 0.847128, 0.528498, 0.49997400000000003, 0.742773, 0.565354, 0.614286, 0.894393, 0.520401, 0.7043950000000001, 0.844501, 0.497868, 0.2656250000000001, 0.944789, 0.512268, 0.2656550000000001, 0.942909, 0.48360000000000003, 0.2765370000000001, 0.8927539999999999, 0.518388, 0.27926300000000004, 0.830071, 0.553099, 0.731553, 0.706645, 0.549092, 0.726051, 0.721575, 0.514243, 0.717881, 0.7340749999999999, 0.463146, 0.7262660000000001, 0.799832, 0.46515599999999996, 0.7